# E-MOSAIC

引用自[E. R. Q. Fernandes, A. C. P. L. F. de Carvalho and X. Yao, "Ensemble of Classifiers Based on Multiobjective Genetic Sampling for Imbalanced Data," in IEEE Transactions on Knowledge and Data Engineering, vol. 32, no. 6, pp. 1104-1115, 1 June 2020, doi: 10.1109/TKDE.2019.2898861.]


## 数据集的预处理 

In [1]:
import numpy as np

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from utils.dataset_utils import get_classes_indexes_counts
import scipy.io as sio  # 从.mat文件中读取数据集
from ucimlrepo import fetch_ucirepo

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

print("#########################加载数据集#########################")
'''
id = :
12 balance_scale  
76 nursery 8、4、12958(4266: 4320: 328: 4044)
'''
# 数据集
# uci_dataset = fetch_ucirepo(id=76)
mat_data = sio.loadmat('../../data/dataset/Satellite.mat')

# 提取变量
# features = uci_dataset.data.features  # 特征数据
# targets = uci_dataset.data.targets  # 标签lable
dataset_x = mat_data['X']
dataset_y = mat_data['Y'][:, 0]  # mat_data['Y']得到的形状为[n,1]，通过[:,0]，得到形状[n,]

# 显示数据集分布
print("特征数据:", dataset_x.shape)
print("label:", dataset_y.shape)
# 统计每个类别的个数，dataset_y.max()+1是类别的个数
classes, counts = get_classes_indexes_counts(dataset_y)  #np.argmax(y_onehot, axis=1)找最大值的索引，将0-1序列转化为0,1,2,3......的整数标签
print("每种类别的分布：", counts)

#############################################划分数据集##################################
# 随机种子
random_seed = 42
print("#########################划分数据集#########################")
x_train, x_test, y_train, y_test = train_test_split(dataset_x, dataset_y, test_size=0.3, random_state=random_seed)

# Standardize the feature data
#scaler = StandardScaler()
#X_train_scaled = scaler.fit_transform(x_train)
#X_test_scaled = scaler.transform(x_test)

# 显示数据集分布
print("特征数据:", x_train.shape)
print("label:", y_train.shape)

# 统计每个类别的分布 np.argmax(y_train, axis=1) Convert one-hot encoded test labels back to single class labels
classes_train, counts_train = get_classes_indexes_counts(y_train)
print("训练集每种类别的分布：", counts_train)

classes_test, counts_test = get_classes_indexes_counts(y_test)
print("测试集每种类别的分布：", counts_test)

print("#########################平衡数据集#########################")
# 确定每个类别的分布
num_instances = int(counts_train.min() * 1.0)  # 向下取整
print("最小数量:", num_instances)

# 在每个类别中随机的选择该数量的实例的索引
balanced_classes = np.array([])
for indexes in classes_train:
    random_selecte_indices = np.random.choice(indexes, size=num_instances, replace=False)
    balanced_classes = np.hstack((balanced_classes, random_selecte_indices))
balanced_classes = np.sort(balanced_classes).astype(int)

# 得到平衡的数据集
balanced_dataset_x = []
balanced_dataset_y = []
for index in balanced_classes:
    balanced_dataset_x.append(x_train[index])
    balanced_dataset_y.append(y_train[index])
balanced_dataset_x = np.array(balanced_dataset_x)
balanced_dataset_y = np.array(balanced_dataset_y).astype(int)

# 显示数据集分布
print("平衡的数据集的特征数据:", balanced_dataset_x.shape)
print("label:", balanced_dataset_y.shape)

# 统计每个类别的分布
classes_balanced_dataset, counts_balanced_dataset = get_classes_indexes_counts(balanced_dataset_y)
print("平衡的数据集中每种类别的分布：", counts_balanced_dataset)

#########################加载数据集#########################
特征数据: (6435, 36)
label: (6435,)
每种类别的分布： [1533  703 1358  626  707 1508]
#########################划分数据集#########################
特征数据: (4504, 36)
label: (4504,)
训练集每种类别的分布： [1083  517  942  425  488 1049]
测试集每种类别的分布： [450 186 416 201 219 459]
#########################平衡数据集#########################
最小数量: 425
平衡的数据集的特征数据: (2550, 36)
label: (2550,)
平衡的数据集中每种类别的分布： [425 425 425 425 425 425]


## 评价函数
（G-mean,mAUC两个目标）

In [2]:
from scipy.stats import gmean
from sklearn.metrics import precision_score, roc_auc_score, accuracy_score
from scipy.stats import mode


##########################由个体得到选择的实例子集的索引###########################
def get_indices(individual):
    '''
    :param individual: individual（用实值进行编码）
    :return: 被选择实例的索引
    '''
    individual = np.round(individual)  # 数据范围在0-1之间，转化成int的同时会舍去小数部分，从而将个体映射到0-1编码
    indices = np.where(individual == 1)  # 1代表选择该实例，返回值是tuple，tuple[0]取元组中的第一个元素
    return indices[0]


###########################获取实例子集############################
def get_subset(individual):
    '''
    :param individual: 
    :return: 实例子集
    '''
    indices = get_indices(individual)
    x_sub = balanced_dataset_x[indices, :]
    y_sub = balanced_dataset_y[indices]
    return x_sub, y_sub


##########################适应度函数（PPV和PFC，为主要、次要指标）#################################
def fitness_function(individual):
    # 使用训练数据进行预测
    ind_pred = individual.mlp.predict(x_test)  # 计算Gmean
    index_pred_proba = individual.mlp.predict_proba(x_test)  # 计算mAUC
    ######################G-mean#########################
    # 计算混淆矩阵
    cm = confusion_matrix(y_test, ind_pred)
    # 计算每类召回率（每类正确预测个数 / 该类总数）
    recall_per_class = cm.diagonal() / cm.sum(axis=1)
    # 计算G-Mean
    geometric_mean = gmean(recall_per_class)
    ########################mAUC#######################
    # 计算 ROC-mAUC（ovo+macro）
    auc_ovo_macro = roc_auc_score(y_test, index_pred_proba, multi_class="ovo",
                                  average="macro")  # index_pred_proba的概率之和应为1
    return round(geometric_mean, 4), round(auc_ovo_macro, 4)


# 集成分类器的投票
def vote_ensembles(save_ensembles, show=False):
    y_pred_ensembles = []
    for ensemble in save_ensembles:
        ind_pred = ensemble.predict(x_test)  # 计算G-mean
        y_pred_ensembles.append(ind_pred)
    # 按列投票，取每列中出现次数最多的类别作为最终分类结果
    final_pred_result = mode(y_pred_ensembles, axis=0, keepdims=False).mode.flatten()
    cm = confusion_matrix(y_test, final_pred_result)
    # 计算每类召回率（每类正确预测个数 / 该类总数）
    recall_per_class = cm.diagonal() / cm.sum(axis=1)
    # 计算Gmean
    geometric_mean = gmean(recall_per_class)

    # 计算准确率
    #accuracy = accuracy_score(y_test, final_pred_result)
    #print(f'Accuracy: {accuracy:.2f}')

    # 打印分类报告
    # print("Classification Report:")
    # print(classification_report(y_test, final_pred_result))

    # 打印混淆矩阵
    # print("Confusion Matrix:")
    # print(confusion_matrix(y_test, final_pred_result))
    return geometric_mean


# 在种群中找到重复的个体
def find_duplicates(pop, similar=0.9):
    """
    找到重复个体的索引。
    :param arrays: 一个包含 array.array 的列表
    :param threshold: 重复的判断阈值
    :return: 重复对的索引列表
    """
    n = len(pop)
    duplicates = []  # 用于记录重复对的索引
    for i in range(n):
        duplicate = ()
        for j in range(i + 1, n):
            # 当前两组数组
            a = pop[i]
            b = pop[j]
            # 计算1的个数
            ones_a = sum(a)
            ones_b = sum(b)
            # 如果其中一个数组全是0，不可能满足条件
            if ones_a == 0 or ones_b == 0:
                continue
            # 计算交集中的1的数量
            common_ones = sum(x == 1 & y == 1 for x, y in zip(a, b))
            # 判断是否满足重复的定义
            if (common_ones / ones_a > similar) and (common_ones / ones_b > similar):
                duplicate = duplicate + (j,)
        duplicates.append(duplicate)
    return duplicates


# 根据索引对，去除种群中重复的个体
def remove_duplicates(pop, duplicates):
    """
    移除重复的个体。
    :param arrays: 一个包含 array.array 的列表
    :param duplicates: 重复对的索引列表
    :return: 去重后的列表
    """
    # 找到所有需要移除的索引
    to_remove = set()  # 只保留后出现的索引
    for duplicate in duplicates:
        to_remove.update(duplicate)  # update是用来更新set集合的
    # 构造去重后的列表
    return [pop[i] for i in range(len(pop)) if i not in to_remove], len(to_remove)

## NDGA-II

In [3]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from instance_selection.nsga_2.genetic_operator import selNSGA2, mutate_binary_inversion, selTournamentDCD
from sklearn.neural_network import MLPClassifier
import array
import random
import matplotlib.pyplot as plt
from deap import base, creator, tools
import warnings

warnings.filterwarnings("ignore")  # 忽略警告
# 最大化评价目标
creator.create("FitnessMaxAndMax", base.Fitness, weights=(1.0, 1.0))
'''
fitness:适应度：Gmean和mAUC
pfc：每个分类器的成对故障信用，用于评估分类器集合的多样性
'''
creator.create("Individual", array.array, typecode='i', fitness=creator.FitnessMaxAndMax, pfc=None, mlp=None)
toolbox = base.Toolbox()

# 设置参数
NDIM = num_instances
lambda_ = 1.2  # 指数分布的参数λ（lambda）在下面的函数中，该值越大越偏向于1
threshold = 1.0  # 阈值（阈值决定了生成0或1）


# 指数分布
def exponential_distribution(lambda_, threshold):
    '''
    :param lambda_: 指数分布的参数λ（lambda）
    :param threshold: 阈值（阈值决定了生成0或1）
    :return: 
    '''
    # 生成一个指数分布的随机数
    value = random.expovariate(lambda_)
    # 根据值与阈值的比较，生成 0 或 1
    if value < threshold:
        return 1
    else:
        return 0


# 二进制编码
toolbox.register("attr_binary", exponential_distribution, lambda_, threshold)  # 0-1编码
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_binary, n=num_instances)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness_function)

# 单点交叉
toolbox.register("mate", tools.cxOnePoint)
# 二进制突变
toolbox.register("mutate", mutate_binary_inversion)
# NSGA-II选择
toolbox.register("select", selNSGA2, x_test=x_test, y_test=y_test)
# 找到种群中重复个体的索引对
toolbox.register("find_duplicates", find_duplicates)
# 去重
toolbox.register("remove_duplicates", remove_duplicates)


# 绘制Pareto Front曲线
def plot_front(fronts, gen, title, show=False):
    """绘制当前代非支配排序的第一等级前沿"""
    fitnesses = [ind.fitness.values for ind in fronts]
    plt.scatter(*zip(*fitnesses), marker='o', label=f"Generation {gen}")
    plt.title(title)
    plt.xlabel("Gmean")
    plt.ylabel("mAUC")
    plt.legend()
    plt.grid()
    if show is True:
        plt.show()
    plt.close()

## 种群的迭代

In [4]:
def main(seed=None):
    random.seed(seed)
    NGEN = 100  # 迭代次数
    POPSIZE = 40  # 种群数量
    CXPB = 1.0  # 交叉因子/交叉率
    MR = 0.2  # 突变因子/突变率
    SIMILAR = 0.9  # 相似度
    learning_rate = 0.1  # MLP的学习率
    hidden_layer_sizes = (15,)
    max_iter = 100
    ####################################迭代过程的记录#############################
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    logbook = tools.Logbook()
    logbook.header = "gen", "evals", "fronts", "duplicates", "Gmean"
    ####################################种群的初始化###########################
    pop = toolbox.population(n=POPSIZE)
    ####################################计算初始种群的适应度###########################
    stop_sign = 0
    ensembles = []  # 当前每个个体对应的mlp模型
    save_ensembles = []  # 保存最好的集成模型
    current_ensembles = []  # 存储当前种群对应的集成模型
    g_means_record = []
    #pop_x_sub = []  # 当前每个个体的实例选择的特征数据
    #pop_y_sub = []  # 当前每个个体对应的实例选择的lable

    # 对于每个个体都训练得到一个mlp模型
    for i in range(len(pop)):
        mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, max_iter=max_iter, random_state=random_seed,
                            learning_rate_init=learning_rate)
        x_sub, y_sub = get_subset(pop[i])
        mlp.fit(x_sub, y_sub)
        ensembles.append(mlp)
        #pop_x_sub.append(x_sub)
        #pop_y_sub.append(y_sub)
        pop[i].mlp = mlp

    # 由mlp模型得到个体的适应度
    for i in range(len(pop)):
        pop[i].fitness.values = toolbox.evaluate(pop[i])
    #################################计算PFC并进行非支配排序#########################################
    # 计算PFC并进行非支配排序 PFC代替拥挤距离在不同的等级中进行二次排序
    pop, pareto_fronts = toolbox.select(pop, len(pop))
    # 保存第一个pareto_front中的模型，进行集成
    for ind in pop:
        save_ensembles.append(ind.mlp)
    g_mean = vote_ensembles(save_ensembles)
    g_means_record.append(g_mean)
    # record = stats.compile(pop)
    # logbook.record(gen=0, evals=len(pop), **record)
    # print(logbook.stream)

    ####################################种群的迭代#################################################
    for gen in range(1, NGEN + 1):
        # 清空当前的集成分类器
        current_ensembles.clear()
        # 选择
        offspring = selTournamentDCD(pop, POPSIZE)
        offspring = [toolbox.clone(ind) for ind in offspring] #clone函数不会复制这些个体的fitness值
        # 交叉
        for i in range(0, len(offspring) - 1, 2):
            if random.random() <= CXPB:
                offspring[i], offspring[i + 1] = toolbox.mate(offspring[i], offspring[i + 1])
            # 突变
            offspring[i] = toolbox.mutate(offspring[i], MR)[0]
            offspring[i + 1] = toolbox.mutate(offspring[i + 1], MR)[0]
            del offspring[i].fitness.values, offspring[i + 1].fitness.values

        # 计算新的种群适应度 
        ensembles.clear()
        # pop_x_sub.clear()
        # pop_y_sub.clear()
        for i in range(len(offspring)):
            mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, max_iter=max_iter, random_state=random_seed,
                                learning_rate_init=learning_rate)
            x_sub, y_sub = get_subset(offspring[i])
            mlp.fit(x_sub, y_sub)
            ensembles.append(mlp)
            #pop_x_sub.append(x_sub)
            #pop_y_sub.append(y_sub)
            offspring[i].mlp = mlp
        for i in range(len(offspring)):
            offspring[i].fitness.values = toolbox.evaluate(offspring[i])

        # 种群的合并
        new_pop = pop + offspring

        # 找到重复个体的索引对
        duplicates = toolbox.find_duplicates(new_pop, SIMILAR)
        # 去除对应重复的个体
        new_pop, num_duplicates = toolbox.remove_duplicates(new_pop, duplicates)
        if len(new_pop) < POPSIZE:
            ###########################################再次对子代进行突变################################################
            for ind in range(len(offspring)):
                # 突变
                offspring[i] = toolbox.mutate(offspring[i], MR)[0]
                del offspring[i].fitness.values
            # 计算新的种群适应度 
            ensembles.clear()
            # pop_x_sub.clear()
            # pop_y_sub.clear()
            for i in range(len(offspring)):
                mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, max_iter=max_iter, random_state=random_seed,
                                    learning_rate_init=learning_rate)
                x_sub, y_sub = get_subset(offspring[i])
                mlp.fit(x_sub, y_sub)
                ensembles.append(mlp)
                # pop_x_sub.append(x_sub)
                # pop_y_sub.append(y_sub)
                offspring[i].mlp = mlp
            for i in range(len(offspring)):
                offspring[i].fitness.values = toolbox.evaluate(offspring[i])
        else:
            offspring.clear()
        # 种群的合并
        pop = new_pop + offspring
        ###############################################得到pareto_fronts############################################
        pop, pareto_fronts = toolbox.select(pop, POPSIZE)
        # print("下一代种群的规模：", len(pop))
        plot_front(pareto_fronts[0], gen, title="Pareto Front (Current Generation)")
        record = stats.compile(pop)
        fronts_distribution = []
        for fronts in pareto_fronts:
            fronts_distribution.append(len(fronts))
        logbook.record(gen=gen, evals=len(pop), fronts=len(fronts_distribution), duplicates=num_duplicates,
                       Gmean=g_means_record[-1], **record)
        print(logbook.stream)
        # 保存第一个等级里的mlp模型进行集成
        for ind in pop:
            current_ensembles.append(ind.mlp)
        g_mean = vote_ensembles(current_ensembles)
        if g_mean > g_means_record[-1]:  # 当前的gmean与列表最后一个gmean做对比
            save_ensembles = current_ensembles
            g_means_record.append(g_mean)
            stop_sign = 0
        else:
            stop_sign += 1
        if stop_sign == 20:
            break
    return pop, logbook, save_ensembles


if __name__ == "__main__":
    pop, stats, ensembles = main()

    print("##############################集成分类器的预测结果：################################")
    vote_ensembles(ensembles)
    print(len(ensembles))

gen	evals	fronts	duplicates	Gmean
1  	40   	6     	30        	0    
2  	40   	6     	25        	0    
3  	40   	7     	25        	0    
4  	40   	7     	26        	0    
5  	40   	7     	27        	0    
6  	40   	9     	28        	0    
7  	40   	11    	30        	0    
8  	40   	12    	23        	0    


KeyboardInterrupt: 